In [1]:
import json
import os
import pandas as pd
from PIL import Image, ImageDraw
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
with open("TableBank_data/Detection_data/Word/Word.json", "r") as file:
    data = json.load(file)

df_annot = pd.DataFrame(data['annotations'])
df_images = pd.DataFrame(data['images'])
df_images.columns = ['file_name', 'width', 'image_id', 'height', 'license'] # word
print(df_annot.shape, df_images.shape)

df_word = pd.merge(df_images, df_annot, on='image_id')[['image_id', 'file_name', 'height', 'width', 'bbox']]
print(df_word.shape)

(101889, 7) (78399, 5)
(101889, 5)


In [3]:
with open("TableBank_data/Detection_data/Latex/Latex.json", "r") as file:
    data = json.load(file)

df_annot = pd.DataFrame(data['annotations'])
df_images = pd.DataFrame(data['images'])
df_images.columns = ['file_name', 'image_id', 'license', 'width', 'height'] # latex
print(df_annot.shape, df_images.shape)

df_latex = pd.merge(df_images, df_annot, on='image_id')[['image_id', 'file_name', 'height', 'width', 'bbox']]
print(df_latex.shape)

(253817, 7) (200183, 5)
(253817, 5)


In [4]:
df = pd.concat([df_word, df_latex], ignore_index=True)
df.head()

,image_id,file_name,height,width,bbox
0,1,%20Edward%20Dawes%20paper_13.jpg,842,596,"[89, 316, 388, 119]"
1,2,%20Edward%20Dawes%20paper_14.jpg,842,596,"[89, 423, 397, 69]"
2,3,%20Edward%20Dawes%20paper_15.jpg,842,596,"[84, 87, 400, 114]"
3,4,%20Edward%20Dawes%20paper_3.jpg,842,596,"[84, 554, 378, 146]"
4,5,%20Edward%20Dawes%20paper_5.jpg,842,596,"[85, 72, 410, 248]"


In [2]:
with open('data_dir/mixed_experiment/val_images.pickle', 'rb') as f:
    val_images = pickle.load(f)

In [4]:
test_images = [x for x in val_images if len(x) < 15][:1000]

In [8]:
for image in test_images:
    loc_df = df[df['file_name']==image+'.jpg']
    assert len(loc_df) != 0
    
    with open('/home/alexeygrigoriev/Projects/TD/data_dir/test_images/TableBank/Ground_Truth/'+\
              image+'.txt', "w") as f_out:
        for bbox in loc_df['bbox']:
            xmin, ymin, dx, dy = bbox
            xmax = xmin + dx
            ymax = ymin + dy
            (left, right, top, bottom) = (xmin, xmax, ymin, ymax)
            f_out.write(' '.join(['table', str(left), str(top), str(right), str(bottom)]))
            f_out.write('\n')

In [5]:
# copy test_images
for image in test_images:
    im = Image.open('data_dir/mixed_experiment/images/'+image+'.jpg')
    im.save('data_dir/test_images/TableBank/mixed_images/'+image+'.jpg')

assert  sorted([x[:-4] for x in os.listdir('data_dir/test_images/TableBank/dtrans_images/')]) == \
        sorted([x[:-4] for x in os.listdir('data_dir/test_images/TableBank/Ground_Truth/')])